In [76]:
import pandas as pd
column_names = ["meter_id", "floor", "unit", "tenant_name", "reading_current", "reading_previous", "present_time", "8/26/2025", "previous_time", "9", "10", "11"]

# Read the CSV file
df = pd.read_csv("sample_readings.csv", low_memory=False)
df.columns = column_names

# Drop columns that are not needed for downstream processing and rows without a current reading
df.drop(columns=["floor", "8/26/2025", "9", "10", "11"], inplace=True)
df = df[df["reading_current"].notna()].reset_index(drop=True)

# Prepare static dates for building timestamps later
df["present_date"] = "2025/9/28"
df["previous_date"] = "2025/8/26"

print(df.head())



                meter_id    unit               tenant_name  reading_current  \
0  Three/NEO-EM-08851888  Shop 1       QBE Exclusive Lobby          88538.0   
1  Three/NEO-EM-50017916  Shop 1  QBE Exclusive Lobby - AC         212784.0   
2  Three/NEO-EM-17873389  Shop 2             Security Bank          93032.0   
3  Three/NEO-EM-28286903  Shop 2        Security Bank - AC         114435.0   
4  Three/NEO-EM-16871324  Shop 3                  7 Eleven         430497.0   

   reading_previous present_time previous_time present_date previous_date  
0           86714.0        1020H       11:18am    2025/9/28     2025/8/26  
1          208391.0        1009H       10:14am    2025/9/28     2025/8/26  
2           92135.0        1020H       11:18am    2025/9/28     2025/8/26  
3          112694.0        1009H       1014:am    2025/9/28     2025/8/26  
4          421106.0        1020H       11:20am    2025/9/28     2025/8/26  


In [77]:
def normalize_time(value: str) -> str | None:
    """Convert raw HHMM-like strings (e.g. 1020H) to HH:MM format with am/pm.

    Returns None when the input cannot be parsed so downstream parsing
    can treat it as missing data."""
    if value is None:
        return None
    text = str(value).strip()
    if not text or text.lower() in {"na", "nan", "none"}:
        return None
    digits = "".join(ch for ch in text if ch.isdigit())
    if len(digits) == 3:
        digits = "0" + digits
    if len(digits) != 4:
        return None
    hours = int(digits[:2]) % 24
    minutes = int(digits[2:]) % 60
    suffix = "am" if hours < 12 else "pm"
    display_hour = hours % 12 or 12
    return f"{display_hour:02d}:{minutes:02d}{suffix}"

present_series = pd.Series(df["present_time"], dtype="string").map(normalize_time)
previous_series = pd.Series(df["previous_time"], dtype="string").map(normalize_time)

df.loc[:, "present_time_clean"] = present_series
df.loc[:, "previous_time_clean"] = previous_series

# Build timestamp columns using the cleaned time values
df["present_timestamp"] = pd.to_datetime(
    df["present_date"] + " " + df["present_time_clean"],
    format="%Y/%m/%d %I:%M%p",
    errors="coerce",
)
df["previous_timestamp"] = pd.to_datetime(
    df["previous_date"] + " " + df["previous_time_clean"],
    format="%Y/%m/%d %I:%M%p",
    errors="coerce",
)

# Create a new column for the month of the reading based on the present timestamp
df["month"] = pd.Series(df["present_timestamp"]).dt.month

print(df[["present_time", "present_time_clean", "present_timestamp"]].head())



  present_time present_time_clean   present_timestamp
0        1020H            10:20am 2025-09-28 10:20:00
1        1009H            10:09am 2025-09-28 10:09:00
2        1020H            10:20am 2025-09-28 10:20:00
3        1009H            10:09am 2025-09-28 10:09:00
4        1020H            10:20am 2025-09-28 10:20:00


In [78]:
#now we want to extract a new dataframe with the following column
df_1 = df[["meter_id", "unit", "tenant_name", "reading_current", "present_timestamp", "month"]]
df_1.columns = ["meter_id", "unit", "tenant_name", "reading", "timestamp", "month"]

#now we want to extract a new dataframe with the following columns
df_2 = df[["meter_id", "unit", "tenant_name", "reading_previous", "previous_timestamp", "month"]]
df_2.columns = ["meter_id", "unit", "tenant_name", "reading", "timestamp", "month"]

result = pd.concat([df_1, df_2])
result.head()


,meter_id,unit,tenant_name,reading,timestamp,month
0,Three/NEO-EM-08851888,Shop 1,QBE Exclusive Lobby,88538.0,2025-09-28 10:20:00,9.0
1,Three/NEO-EM-50017916,Shop 1,QBE Exclusive Lobby - AC,212784.0,2025-09-28 10:09:00,9.0
2,Three/NEO-EM-17873389,Shop 2,Security Bank,93032.0,2025-09-28 10:20:00,9.0
3,Three/NEO-EM-28286903,Shop 2,Security Bank - AC,114435.0,2025-09-28 10:09:00,9.0
4,Three/NEO-EM-16871324,Shop 3,7 Eleven,430497.0,2025-09-28 10:20:00,9.0


In [79]:
result.to_csv("recording_sample.csv", index=False)

In [81]:
from pathlib import Path

rec_path = Path('recording_sample.csv')
query_path = Path('query_1-2025-11-13_93126.csv')
output_path = Path('meter_matches.csv')

rec = pd.read_csv(rec_path)
query = pd.read_csv(query_path)

rec['meter_suffix'] = rec['meter_id'].astype(str).str.extract(r'([0-9]+)$')
query['meter_id_str'] = query['meter_id'].astype(str)

merged = rec.merge(query, left_on='meter_suffix', right_on='meter_id_str', how='inner', suffixes=('_recording', '_query'))

# Reorder columns for clarity
columns = [
    'meter_id_recording', 'meter_suffix', 'meter_id_str', 'meter_ref', 'm_id', 'description',
    'tenant_name_recording', 'tenant_name_query', 'unit_recording', 'unit_number',
    'reading', 'timestamp', 'month', 'tenant_id', 'unit_id'
]


merged['unit_number']= merged['unit_number'].str.replace('NEO', 'NEO3')
merged['tenant_name_recording'] = merged['tenant_name_recording'].str.replace('7 Eleven', '7Eleven')
merged['tenant_name_recording'] = merged['tenant_name_recording'].str.replace('QBE Exclusive Lobby', 'QBE Group Shared Services Ltd')
merged['tenant_name_recording'] = merged['tenant_name_recording'].str.replace('QBE/Tyler Technologies Philippines./Acquire', 'QBE Group Shared Services Ltd')
merged['tenant_name_query'] = merged['tenant_name_query'].str.replace('Lobby', '').str.replace('(vacant)', 'vacant').str.strip('-')
merged['tenant_name_recording'] = merged['tenant_name_recording'].str.replace('Finace', 'Finance')
merged['tenant_name_recording'] = merged['tenant_name_recording'].str.replace('.', '').str.title().str.strip()
merged['tenant_name_query'] = merged['tenant_name_query'].str.replace('.', '').str.title().str.strip()





merged['description'] = merged.apply(lambda row: row['tenant_name_recording'].replace(row['tenant_name_query'],'').strip('(').strip(')'), axis=1)
merged['description'] = merged['description'].str.replace('-', '').str.replace('(', '').str.replace(')', '').str.strip()
existing_cols = [c for c in columns if c in merged.columns]
merged.to_csv(output_path, index=False, columns=existing_cols)
print(f"Wrote {len(merged)} rows to {output_path}")
merged.head(20)


Wrote 108 rows to meter_matches.csv


,meter_id_recording,unit,tenant_name_recording,reading,timestamp,month,meter_suffix,tenant_name_query,tenant_id,unit_number,unit_id,meter_id_query,meter_ref,m_id,meter_id_str,description
0,Three/NEO-EM-50017916,Shop 1,Qbe Group Shared Services Ltd - Ac,212784.0,2025-09-28 10:09:00,9.0,50017916,Qbe Group Shared Services Ltd -,17,NEO3_GF01,41,50017916,50017916,35,50017916,Ac
1,Three/NEO-EM-17873389,Shop 2,Security Bank,93032.0,2025-09-28 10:20:00,9.0,17873389,Security Bank,20,NEO3_GF02,42,17873389,17873389,80,17873389,
2,Three/NEO-EM-28286903,Shop 2,Security Bank - Ac,114435.0,2025-09-28 10:09:00,9.0,28286903,Security Bank,20,NEO3_GF02,42,28286903,28286903,36,28286903,Ac
3,Three/NEO-EM-16871324,Shop 3,7Eleven,430497.0,2025-09-28 10:20:00,9.0,16871324,7Eleven,10,NEO3_GF03,43,16871324,16871324,81,16871324,
4,Three/NEO-EM-14864628,7-1,Lbs Digital Infrastructure Corp - Ldic (Lighti...,188425.0,2025-09-28 10:06:00,9.0,14864628,Red Fu,18,NEO3_0702,36,14864628,14864628,39,14864628,Lbs Digital Infrastructure Corp Ldic Lighting...
5,Three/NEO-EM-14864257,7-1,Lbs Digital Infrastructure Corp - Ldic (Accu),208084.0,2025-09-28 10:13:00,9.0,14864257,Lbs Digital Infrastructure Corp - Ldic,14,NEO3_0701,35,14864257,14864257,29,14864257,Accu
6,Three/NEO-EM-28693813,7-2,Red Fu (Lighting & Power),1018483.0,2025-09-28 10:06:00,9.0,28693813,Lbs Digital Infrastructure Corp - Ldic,14,NEO3_0701,35,28693813,28693813,38,28693813,Red Fu Lighting & Power
7,Three/NEO-EM-14864627,7-2,Red Fu (Accu),110412.0,2025-09-28 10:06:00,9.0,14864627,Red Fu,18,NEO3_0702,36,14864627,14864627,30,14864627,Accu
8,Three/NEO-EM-16870422,8-1,"Adventure International Tours, Inc (Aiti) (Lig...",234128.0,2025-09-28 10:05:00,9.0,16870422,"Adventure International Tours, Inc (Aiti)",11,NEO3_0801,37,16870422,16870422,40,16870422,Lighting & Power
9,Three/NEO-EM-28286864,8-1,"Adventure International Tours, Inc (Aiti) (Lig...",181415.0,2025-09-28 10:13:00,9.0,28286864,"Adventure International Tours, Inc (Aiti)",11,NEO3_0801,37,28286864,28286864,31,28286864,Lighting & Power
